In [ ]:
import cv2
from deepface import DeepFace
import mediapipe as mp

# Carregar o Classificador Haar Cascade para detecção de rostos
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Inicializar MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.3)

# Inicializar variáveis para rastrear informações
frame_count = 0
face_count = 0
emotion_count = {}
activity_count = {'standing': 0, 'sitting': 0, 'dancing': 0, 'laying': 0}

# Inicializar contadores de anomalia
face_anomaly_count = 0
activity_anomaly_count = 0

# Carregar vídeo
cap = cv2.VideoCapture('video_analise.mp4')

# Definir codec e criar objeto VideoWriter para o vídeo de saída
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_video.mp4', fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

# Função para determinar atividade com base nos pontos de referência da pose
def determine_activity(results):
    if results.pose_landmarks:
        left_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
        left_hip = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP]
        
        if left_shoulder.visibility > 0.5 and left_hip.visibility > 0.5:
            shoulder_y = left_shoulder.y
            hip_y = left_hip.y
            # Determinar se está sentado ou em pé com base nas coordenadas Y
            if hip_y > shoulder_y:
                return 'sitting'
            else:
                return 'standing'
    return 'unknown'

# Iterar por cada quadro do vídeo
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    front_faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    for (x, y, w, h) in front_faces:
        # Analisar emoção na região do rosto detectada
        face_image = frame[y:y + h, x:x + w]
        
        try:
            analysis = DeepFace.analyze(face_image, actions=['emotion'], enforce_detection=False)
            if analysis and isinstance(analysis, list) and len(analysis) > 0:
                dominant_emotion = analysis[0]['dominant_emotion']
                emotion_count.setdefault(dominant_emotion, 0)
                emotion_count[dominant_emotion] += 1
                face_count += 1
            else:
                face_anomaly_count += 1  # Incrementar contagem de anomalias se não houver análise válida
        except Exception as e:
            print(f"Erro na detecção de emoção: {e}")
            face_anomaly_count += 1  # Incrementar contagem de anomalias em caso de erro

        # Desenhar retângulo e colocar texto para a emoção
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, dominant_emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

    # Analisar pose para detecção de atividade
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    # Determinar atividade com base nos pontos de referência da pose
    activity = determine_activity(results)
    if activity != 'unknown':
        activity_count[activity] += 1
    else:
        activity_anomaly_count += 1  # Incrementar contagem de anomalias para detecção de atividade

    # Escrever atividade no vídeo
    cv2.putText(frame, activity, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    # Escrever o quadro no vídeo de saída
    out.write(frame)

# Liberar recursos
cap.release()
out.release()

# Imprimir e salvar relatório de análise
report = (
    f"Total de quadros: {frame_count}\n"
    f"Rostos detectados: {face_count}\n"
    f"Anomalias na detecção de rostos: {face_anomaly_count}\n"
    f"Emoções detectadas:\n" +
    ''.join([f"{emotion}: {count}\n" for emotion, count in emotion_count.items()]) +
    f"Atividades detectadas:\n" +
    ''.join([f"{activity}: {count}\n" for activity, count in activity_count.items()]) +
    f"Anomalias na detecção de atividades: {activity_anomaly_count}\n"
)

# Imprimir o relatório no console
print(report)

# Escrever o relatório em um arquivo de texto
with open('analysis_report.txt', 'w') as f:
    f.write(report)

Total de quadros: 3326
Rostos detectados: 1177
Anomalias na detecção de rostos: 0
Emoções detectadas:
fear: 137
surprise: 179
sad: 76
happy: 527
neutral: 247
angry: 11
Atividades detectadas:
standing: 71
sitting: 1118
dancing: 0
laying: 0
Anomalias na detecção de atividades: 2137

